In [15]:
import pandas as pd
import yaml

# where you want to use regular 28x28 MNIST or 20x20 version
dataset = "mnist" 
#dataset = "fashion_mnist" 


In [16]:
if dataset == "mnist":
    first_in_dim = 400
    filename = "mnist_config_20x20.yaml"
    df = pd.read_csv("mnist_hyperparameters.csv")
elif dataset == "fashion_mnist":
    first_in_dim = 784
    filename = "fashion_mnist_config_28x28.yaml"
    df = pd.read_csv("fashion_mnist_hyperparameters.csv")

# Convert the DataFrame to a list of dictionaries
models = df.to_dict(orient="records")

# Create the YAML structure
yaml_structure = {"models": {}}

# Function to round the number to the nearest multiple of the output size
def round_to_nearest_multiple(value, multiple):
    return multiple * round(value / multiple)

# Populate the YAML structure with models
for i, model in enumerate(models, start=1):
    # Zero-pad model names to 3 digits 
    model_name = f"model_{str(i).zfill(3)}"
    layers_config = {}
    
    for layer in range(1, model["H"] + 1):
        # Zero-pad layer names to 3 digits
        layer_name = f"LogicLayer{str(layer).zfill(3)}"
        
        # Adjust in_dim to the nearest multiple of 10
        in_dim = first_in_dim if layer == 1 else round_to_nearest_multiple(model["W"], 10)
        
        # Adjust out_dim to the nearest multiple of 10
        out_dim = round_to_nearest_multiple(model["W"], 10)
        
        layers_config[layer_name] = {
            "in_dim": in_dim,
            "out_dim": out_dim,
            "device": "cuda",
            "implementation": "cuda",
            "connections": "random",
            "grad_factor": 1, # we can try different grad_factor values as well
        }
    
    yaml_structure["models"][model_name] = {
        "input_dim": first_in_dim, 
        "output_size": 10, # for MNIST classification
        "tau": model["tau"],
        "learning_rate": model["lr"],
        "layers_config": layers_config,
        # add the AR parameter in here 
    }

# Save to a YAML file
with open(f'{filename}', "w") as file:
    yaml.dump(yaml_structure, file, default_flow_style=False)

print(f"YAML file '{filename}' generated successfully.")

YAML file 'mnist_config_20x20.yaml' generated successfully.
